<a href="https://colab.research.google.com/github/raveinnovate/DATA304-Project/blob/main/baltimore_mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import folium
import pandas as pd
import requests
import json

In [ ]:
geo_json_data = requests.get(
    "https://raw.githubusercontent.com/python-visualization/folium-example-data/main/us_states.json"
).json()

In [4]:
with open('/content/drive/MyDrive/data304_childrens_program/data/zipcodes.json', 'r') as file:
    zip = json.load(file)

In [70]:
unique_schools = [{"index":0,"school_name":"Charles Carroll Barrister Elementary","address":"1327 Washington Boulevard Baltimore, MD 21230","zipcode":21230},{"index":48,"school_name":"George Washington Elementary","address":"800 Scott Street Baltimore, MD 21230","zipcode":21230}, {"index":99,"school_name":"Southwest Baltimore Charter","address":"1300 Herkimer Street Baltimore, MD 21223","zipcode":21223}]

# Dataframe of Paul's Place Serviced Schools
pp_schools = pd.DataFrame(unique_schools)
pp_schools

,index,school_name,address,zipcode
0,0,Charles Carroll Barrister Elementary,"1327 Washington Boulevard Baltimore, MD 21230",21230
1,48,George Washington Elementary,"800 Scott Street Baltimore, MD 21230",21230
2,99,Southwest Baltimore Charter,"1300 Herkimer Street Baltimore, MD 21223",21223


In [73]:
url = "https://reportcard.msde.maryland.gov/Graphs/#/ReportCards/ReportCardSchool/1/EM/1/30/0034/0"
# pp_schools["schoolID"] = "0034"

pp_schools.loc[0, "schoolID"] = "0034"
pp_schools.loc[1, "schoolID"] = "0022"
pp_schools.loc[2, "schoolID"] = "0328"
pp_schools

,index,school_name,address,zipcode,schoolID
0,0,Charles Carroll Barrister Elementary,"1327 Washington Boulevard Baltimore, MD 21230",21230,0034
1,48,George Washington Elementary,"800 Scott Street Baltimore, MD 21230",21230,0022
2,99,Southwest Baltimore Charter,"1300 Herkimer Street Baltimore, MD 21223",21223,0328


In [80]:
# Create report card url column for maryland schools
rc_url = "https://reportcard.msde.maryland.gov/Graphs/#/ReportCards/ReportCardSchool/1/EM/1/30/"

for i in range(len(pp_schools)):
  for id in pp_schools.schoolID:
    pp_schools.loc[i, "reportCardUrl"] = rc_url + id


# "latitude": [
#         39.27851,
#         39.283891,
#         39.28179]


# "longitude": [
#     -76.637289,
#     -76.628641,
#     -76.63856]

pp_schools["latitude"] = [
    -76.637289,
    -76.628641,
    -76.63856]


pp_schools["longitude"] =  [
        39.27851,
        39.283891,
        39.28179]

pp_schools

,index,school_name,address,zipcode,schoolID,reportCardUrl,latitude,longitude
0,0,Charles Carroll Barrister Elementary,"1327 Washington Boulevard Baltimore, MD 21230",21230,0034,https://reportcard.msde.maryland.gov/Graphs/#/...,-76.637289,39.278510
1,48,George Washington Elementary,"800 Scott Street Baltimore, MD 21230",21230,0022,https://reportcard.msde.maryland.gov/Graphs/#/...,-76.628641,39.283891
2,99,Southwest Baltimore Charter,"1300 Herkimer Street Baltimore, MD 21223",21223,0328,https://reportcard.msde.maryland.gov/Graphs/#/...,-76.638560,39.281790


In [111]:
urlList = list(pp_schools.reportCardUrl)
urlList[0]

'https://reportcard.msde.maryland.gov/Graphs/#/ReportCards/ReportCardSchool/1/EM/1/30/0328'

In [115]:
import math
import branca
import folium
from folium import IFrame
from folium.features import Popup

# Baltimore coordinates
center_lat = 39.2904
center_lon = -76.6122
mile_radius = 10

# 1 degree latitude ≈ 69 miles
lat_offset = mile_radius / 69

# 1 degree longitude ≈ 69 * cos(latitude) miles
lon_offset = mile_radius / (69 * math.cos(math.radians(center_lat)))

# Compute bounds
min_lat = center_lat - lat_offset
max_lat = center_lat + lat_offset
min_lon = center_lon - lon_offset
max_lon = center_lon + lon_offset

print(f"min_lat: {min_lat}")
print(f"max_lat: {max_lat}")
print(f"min_lon: {min_lon}")
print(f"max_lon: {max_lon}")



m = folium.Map(
    max_bounds=True,
    location=[39.2904, -76.6122],
    zoom_start=14,
    min_lat=min_lat,
    max_lat=max_lat,
    min_lon=min_lon,
    max_lon=max_lon,
)

folium.CircleMarker([max_lat, min_lon], tooltip="Upper Left Corner").add_to(m)
folium.CircleMarker([min_lat, min_lon], tooltip="Lower Left Corner").add_to(m)
folium.CircleMarker([min_lat, max_lon], tooltip="Lower Right Corner").add_to(m)
folium.CircleMarker([max_lat, max_lon], tooltip="Upper Right Corner").add_to(m)


folium.GeoJson(zip).add_to(m)

popup = folium.GeoJsonPopup(fields=["zipname", "zipcode1"])

folium.GeoJson(
    zip,
    highlight_function=lambda feature: {
        "color": "purple",
        "fillColor": (
            "darkpurple" if "21230" in feature["properties"]["zipcode1"].lower() else "white"
            "darkpurple" if "21223" in feature["properties"]["zipcode1"].lower() else "white"
        ),
    },
    popup=popup,
    popup_keep_highlighted=True,
).add_to(m)

# iframe = IFrame(f'<iframe src="{urlList[0]}" width="400" height="300"></iframe>', width=420, height=320)

# # Create a popup and add the iframe
# popup = Popup(iframe, max_width=800)

# # Add a marker to the map with the popup
# folium.Marker([39.28179, -76.63856], popup=popup).add_to(m)

# Add Paul's Place Schools to Map
for lat, long in pp_schools[["latitude", "longitude"]].values:
  print(lat,long)
  folium.Marker([long, lat]).add_to(m)


m

min_lat: 39.145472463768115
max_lat: 39.43532753623188
min_lon: -76.79945786524658
max_lon: -76.42494213475342
-76.637289 39.27851
-76.628641 39.283891
-76.63856 39.28179
